<a href="https://colab.research.google.com/github/DiogoNeves/tool_usage_example/blob/main/Tool_Usage_Web_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anthropic

In [30]:
!pip install duckduckgo_search

In [2]:
from typing import Any
from collections import namedtuple

In [3]:
import anthropic
from google.colab import userdata

ANTHROPIC_KEY = userdata.get("ANTHROPIC_API_KEY")
MODEL = "claude-3-haiku-20240307"

client = anthropic.Anthropic(api_key=ANTHROPIC_KEY)

In [49]:
SYSTEM_MESSAGE = ("You are a useful assistant that can answer general questions"
                  " from users. Some questions require up-to-date knowledge,"
                  " for which you should search the web with the tools"
                  " provided. When returning answers from web searches, provide"
                  " the sources as markdown reference links close to where the"
                  " answer shows. For example, this is a reference link"
                  " [1](https://reference.com)")

TOOLS = [
    {
        "name": "search_web",
        "description": ("Search the web based on a search query provided by the"
                        " user. The query should be optimised for search"
                        " engines to find the information needed to answer the"
                        " main user query."),
        "input_schema": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": ("A web search optimised search query.")
                }
            },
            "required": ["query"],
        },
    },
]

In [48]:
from duckduckgo_search import DDGS

NUM_RESULTS_TO_CONSIDER = 3


def search_web(query: str) -> list[str]:
    web_results = DDGS().text(query, max_results=NUM_RESULTS_TO_CONSIDER)
    print("web results: ", web_results)
    return [_parse_web_result(result) for result in web_results]


def _parse_web_result(web_result: dict) -> str:
    return f"""<web_result>
                <title>{web_result["title"]}</title>
                <body>{web_result["body"]}</body>
                <source>{web_result["href"]}</source>
               </web_result>"""


FUNCTION_MAP = {
    "search_web": search_web
}

In [44]:
def run_tool(tool_block: Any) -> dict:
    """Run the right tool to the tool_block."""
    print(f"Called out to {tool_block.name}")

    tool_to_run = FUNCTION_MAP.get(tool_block.name, None)
    assert tool_to_run is not None

    tool_id = tool_block.id
    result = tool_to_run(**tool_block.input)
    return {
        "type": "tool_result",
        "tool_use_id": tool_id,
        "content": str(result)
    }


def query_claude(user_prompt: str | list, conversation: list) -> tuple[str, list]:

    messages = conversation + [{"role": "user", "content": user_prompt}]

    message = client.beta.tools.messages.create(
        model=MODEL,
        max_tokens=1000,
        temperature=0.0,
        system=SYSTEM_MESSAGE,
        tools=TOOLS,
        messages=messages
    )

    assistant_turn = {"role": "assistant", "content": message.content}
    new_conversation = messages + [assistant_turn]

    tool_blocks = [block for block in message.content
                   if block.type == "tool_use"]

    if not tool_blocks:
        return "".join([block.text for block in message.content]), \
            new_conversation

    print("Tools called: ", message.content)

    tool_results = []
    for block in tool_blocks:
        tool_result = run_tool(block)
        tool_results.append(tool_result)

    return query_claude(tool_results, new_conversation)

## Searching the web with Claude

### Examples to test
**Positive Examples:**  
"What is Android 15's release date"  
"When is the new iPhone coming out"  
"Is Perplexity a publicly traded company"  
"how can I search the web in python without having to setup paid APIs?"  

**Negative Examples:**  
"How do translate coding into japanese"  
"What's your name"  
"How do LLMs work"  
"Say hello"

In [50]:
response, conversation = query_claude("What is Android 15's release date?", [])
print("Conversation: ", conversation)
print("Response: ", response)

Tools called:  [ToolUseBlock(id='toolu_019Tih2q8Mun2RLiMbu3mzqE', input={'query': 'Android 15 release date'}, name='search_web', type='tool_use')]
Called out to search_web
web results:  [{'title': 'Android 15: All the new features, eligible devices, and release date', 'href': 'https://www.androidcentral.com/apps-software/android-15', 'body': 'Android 15 is the next version of Android, codenamed Vanilla Ice Cream, and it is expected to launch in August 2024. Learn about the new features in the first preview build, such as notification cooldown, keyboard vibration, partial screen recording, and more.'}, {'title': 'Android 15: What we know so far and what features we want to see', 'href': 'https://www.androidauthority.com/android-15-features-3401939/', 'body': 'Android 15 is the next major version of the Android platform, with a codename of Vanilla Ice Cream. It is currently in testing for Pixel devices and will likely launch in October 2024 with new UI changes, performance improvements, 

In [51]:
response, conversation = query_claude("How do translate 'coding' into japanese?", [])
print("Conversation: ", conversation)
print("Response: ", response)

Tools called:  [ToolUseBlock(id='toolu_01K9ykoHPMUoW6Y7FuuezNMP', input={'query': "how to translate 'coding' into japanese"}, name='search_web', type='tool_use')]
Called out to search_web
web results:  [{'title': 'Coding languages translator, Code Translator: Convert Between ...', 'href': 'https://developerpal.dev/translate', 'body': 'Discover the future of coding with our revolutionary Translation Tool, designed to bridge the gap between diverse programming languages. Our innovative platform offers a seamless experience in translating code from one programming language to another, enabling developers, students, and technology enthusiasts to effortlessly understand and ...'}, {'title': 'source code - What is programming like in the Japanese language ...', 'href': 'https://softwareengineering.stackexchange.com/questions/46386/what-is-programming-like-in-the-japanese-language', 'body': "Additionally, some older implementations of languages that translate code into tokens (not true compil

In [52]:
response, conversation = query_claude("Say hello", [])
print("Conversation: ", conversation)
print("Response: ", response)

Conversation:  [{'role': 'user', 'content': 'Say hello'}, {'role': 'assistant', 'content': [TextBlock(text='Hello!', type='text')]}]
Response:  Hello!
